In [1]:
from qililab import deserialize_from, serialize_to
from qililab.typings import Parameter

In [2]:
parameters = {"readoutA": {Parameter.IF: 400e6}, "readoutB": {Parameter.IF: 500e6}}

In [3]:
serialize_to(parameters, "./parameters.yml")

In [4]:
loaded_parameters = deserialize_from("./parameters.yml", dict)

In [5]:
loaded_parameters

{'readoutA': {<Parameter.IF: 'intermediate_frequency'>: 400000000.0},
 'readoutB': {<Parameter.IF: 'intermediate_frequency'>: 500000000.0}}

In [ ]:
from qililab import platform

# parameters = loaded_parameters["readoutA"]

chosen_bus = "readout" + "A"
for parameter, value in loaded_parameters.get(chosen_bus).items():
    platform.set_parameter(alias=chosen_bus, parameter=parameter, value=value)

AttributeError: module 'qililab.platform' has no attribute 'set_parameter'

In [ ]:
from qililab import Calibration
from qililab.waveforms import IQPair, Square

my_calibration = Calibration()
my_calibration.add_waveform("readoutA", "measure", Square(1.0, 20))
my_calibration.add_weights("readoutA", "weights", IQPair(I=Square(1.0, 10), Q=Square(1.0, 10)))
my_calibration.add_waveform("readoutB", "measure", Square(0.5, 40))
my_calibration.add_weights("readoutB", "weights", IQPair(I=Square(0.5, 10), Q=Square(0.5, 10)))

my_calibration.save_to("my_calibration.yml")

'my_calibration.yml'

In [ ]:
from qililab import QProgram

amplitude = 1.0
duration = 500

qp = QProgram()
qp.play(bus="readout_bus", waveform="measure")
qp.measure(bus="readout_bus", waveform="measure", weights="weights")
qp.play(bus="readout_bus", waveform=Square("amplitude", "duration"))

In [ ]:
from qililab.qprogram import QbloxCompiler

compiler = QbloxCompiler()

output = compiler.compile(qp, calibration=my_calibration, bus_mapping={"readout_bus": "readoutB"})

In [12]:
output.sequences

{'readoutB': {'waveforms': {'waveform_0': {'data': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'index': 0}, 'waveform_1': {'data': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'index': 1}}, 'weights': {'weight_0': {'data': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'index': 0}}, 'acquisitions': {'acquisition_0': {'num_bins': 1, 'index': 0}}, 'program': 'setup:\n                wait_sync        4              \n                set_mrk          0              \n                upd_param        4              \n\nmain:\n                play             0, 1, 40       \n                play             0, 1, 4        \n                acquire_weighed  0, 0, 0, 0, 10 \n   

In [13]:
from dataclasses import dataclass


@dataclass
class Constants:
    AMPLITUDE: float
    DURATION: int
    AVERAGES: int

In [14]:
my_constants = {
    "readoutA": Constants(AMPLITUDE=1.0, DURATION=500, AVERAGES=1),
    "readoutB": Constants(AMPLITUDE=0.5, DURATION=200, AVERAGES=100),
}

In [15]:
serialize_to(my_constants, "constants.yml")

In [17]:
my_loaded_constants = deserialize_from("constants.yml")

In [21]:
square = Square(my_loaded_constants["readoutA"].AMPLITUDE, 500)
square.amplitude

1.0